In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import  os
os.chdir('/content/drive/MyDrive/sm-embedder')

In [ ]:
import os
import numpy as np
from preprocessing import images_dataset_from_tfrecord
from train import train_head, export_centers
from evaluation import \
        inference, \
        labels_from_embeddings, \
        export_embeddings_for_visualization, \
        export_metadata, \
        export_projector_data, \
        get_results_summary 

build_dir = "/home/duo/sm-embedder/build"

In [ ]:
# The hyperparameters will be defined here for cleanliness. No need to expose them. 
head_model, centers = train_head(os.path.join(build_dir, 'checkpoints/head'),
                                 train_record=os.path.join(build_dir, 'train_featuremaps.tfrecord'),
                                 val_record=os.path.join(build_dir, 'val_featuremaps.tfrecord'),
                                 num_classes=20,
                                 featuremaps_dim=1280,
                                 embedder_dim=32,
                                 centerloss_alpha=0.8,
                                 logs_dir=os.path.join(build_dir, 'logs/'),
                                 batch_size=32,
                                 epochs=10,
                                 steps=1000)

In [ ]:
# Now we export the centers, we will need them later.
export_centers(centers, os.path.join(build_dir, 'centers.tsv'))

In [ ]:
# Here we compute the embeddings for the test.tfrecord dataset. 
test_dataset_embeddings = inference(head_checkpoint=os.path.join(build_dir, 'checkpoints/head'),
                                    dataset_record=os.path.join(build_dir, 'test.tfrecord'),
                                    input_shape=(300, 300, 3)) 

In [ ]:
# Loading the dataset again to get the labels
labels_dataset = images_dataset_from_tfrecord(os.path.join(build_dir, 'test.tfrecord'))
labels_it = labels_dataset.map(lambda x: x['label']).as_numpy_iterator()
real_labels = np.array(list(labels_it))

In [ ]:
# loading centers from tsv file
centers = np.loadtxt(os.path.join(build_dir, 'centers.tsv'))
inferred_labels = labels_from_embeddings(test_dataset_embeddings, centers)

In [ ]:
summary = get_results_summary(inferred_labels, real_labels)
print(summary)

In [ ]:
# Export the centers and the computed embeddings to a single file for visuals 
# The two files you want to load on projector.tensorflow.org are:
#     - inference_embeddings.tsv: contains the the centers and the inferred embeddings
#     - metadata.tsv: contains the labels for each center and embeddings
export_embeddings_for_visualization(
        embeddings=test_dataset_embeddings,
        filename=os.path.join(build_dir, 'inference_embeddings.tsv'),
        centers_filename=os.path.join(build_dir,'centers.tsv'))

export_metadata(os.path.join(build_dir, 'metadata.tsv'), real_labels, num_classes=20)

# Also exporting embeddings data to tensorboard. I don't know why, but the labels
# are not working... you can load them manually from ". (build_dir)/metadata.tsv"
embs_for_projector = np.loadtxt(os.path.join(build_dir, 'inference_embeddings.tsv'))
export_projector_data(
        embs_for_projector,
        os.path.join(build_dir, 'metadata.tsv'),
        os.path.join(build_dir, 'logs/'))

# That's all folks! Thank you for your time an patience! 
# I hope you enjoyed!
# Take a look at the build_dir to see all the artifacts generated.
# Cya!